Esta pauta cuenta con una forma de modelar el problema, existen múltiples maneras distintas de abordarlo y no necesariamente una es mejor que otra, lo que importa es que apliquen la lógica y buenas prácticas de programación.

En la pauta hay uso de sintaxis un poco más avanzada a lo que se espera para efectos de este curso, el objetivo es que la pauta también signifique un aprendizaje para ustedes, no quiere decir que se espere exactamente lo mismo que en la pauta. Ante cualquier duda sobre la pauta pueden ocupar las issues del curso.

In [ ]:
import random
from datetime import datetime, timedelta

In [ ]:
# estas son excepciones propias que cree para organizarme mejor y darme cuenta 
# facilmente de si tenia un error en alguna parte, no es necesario hacerlo.
class NotImplementedError(Exception):
    pass
class CosecharError(Exception):
    pass

In [ ]:
class Plant:
    def __init__(self):
        self.fecha_plantacion: datetime = None
        self.humedad_optima: float = None
        self.humedad_actual: float = 100
        # voy a sumarle un porcentaje de crecimiento, cuando llegue (o supere) 
        # 100, aumento el tamanho
        self.crecer_porcentaje:float = 0.0
        self.cant_cosechada = 0
        return

    def cosechar(self):
      # levanto una de mis excepciones en caso de que se me olvidara implementar
      # este metodo en alguna clase que herede de esta
        raise NotImplementedError

    def agregar_fruto(self, n: int):
        raise NotImplementedError
        
    def crecer(self, luz):
        #voy a crecer un % entre 0.5 y 5 de un fruto cada hora SSI hay luz
        if luz == 0:
            return 
        self.crecer_porcentaje += random.uniform(0.5, 5)
        if self.crecer_porcentaje >= 100:
            self.crecer_porcentaje -= 100
            self.agregar_fruto(1)

    def set_humedad_medida(self, humedad: int):
        self.humedad_actual = humedad

class Lechuga(Plant):
    cls_id = 0
    def __init__(self):
        super().__init__()
        self.n_hojas: int = 0
        self.humedad_optima: float = 70
        self.id = Lechuga.cls_id + 1
        self.tamano = 'chica'
        Lechuga.cls_id += 1

    def fruto(self):
        return 'hoja'

    def set_tamano(self):
        if self.n_hojas < 10:
            self.tamano = 'chica'
        elif self.n_hojas > 40:
            self.tamano = 'grande'
        else:
            self.tamano = 'mediana'

    def cosechar(self, n: int):
        if n <= self.n_hojas:
            self.n_hojas -= n
            self.cant_cosechada += n
            self.set_tamano()
            return n
        else:
            raise CosecharError

    def agregar_fruto(self, n: int):
        self.n_hojas += n
        self.set_tamano()

    def puede_cosechar(self, h=None):
        return self.n_hojas

    def name(self):
        return "Lechuga"

    def full_name(self):
        return self.name()

class Tomate(Plant):
    cls_id = 0
    def __init__(self):
        super().__init__()
        # elijo al azar entre cherry y normal
        self.raza: str = random.sample(['cherry', 'normal', 'normal'], 1)[0]
        self.n_tomatoes: int = 0
        self.acidez: float = None # no sirve de nada, pero se pide en el enunciado.
        self.humedad_optima = 30
        self.id = Tomate.cls_id + 1
        Tomate.cls_id += 1
         
    def fruto(self):
        return 'tomate'

    def cosechar(self, n: int):
        if n <= self.n_tomatoes:
            self.n_tomatoes -= n
            self.cant_cosechada += n
            return n
        else:
            raise CosecharError

    def agregar_fruto(self, n: int):
        self.n_tomatoes += n

    def puede_cosechar(self, h=None):
        if self.raza == 'cherry':
            if h == None or h.hour < 12:
                return self.n_tomatoes
        else:
            return self.n_tomatoes
        return 0

    def name(self):
        return "Tomate"

    def full_name(self):
        return f"Tomate {self.raza}"

class Cilantro(Plant):
    cls_id = 0
    def __init__(self):
        super().__init__()
        self.n_ramas: int = 0
        self.humedad_optima = 65
        self.id = Cilantro.cls_id + 1
        Cilantro.cls_id += 1

    def fruto(self):
        return 'rama'

    def cosechar(self, n: int,):
        if n <= self.n_ramas:
            self.n_ramas -= n
            self.cant_cosechada += n
            return n
        else:
            raise CosecharError

    def agregar_fruto(self, n: int):
        self.n_ramas += n

    def puede_cosechar(self, h=None):
        return self.n_ramas

    def name(self):
        return "Cilantro"

    def full_name(self):
        return self.name()

In [ ]:
class Sensor:
    def __init__(self):
        self.ultima_medicion = None

    def medir():
        raise NotImplementedError

class Hsensor(Sensor):
    def __init__(self):
        super().__init__()
        self.ultima_medicion = {"Lechuga": {}, "Tomate": {}, "Cilantro": {}}
        
    def medir(self, planta):
        # para medir la humeda de una planta, tomo la ultima medicion que le hice a esa planta
        # y le resto un numero entre 1 y 30
        if planta.id not in self.ultima_medicion[planta.name()]:
            self.ultima_medicion[planta.name()][planta.id] = planta.humedad_actual
        elif planta.humedad_actual > self.ultima_medicion[planta.name()][planta.id]:
            self.ultima_medicion[planta.name()][planta.id] = planta.humedad_actual

        disminucion = round(random.uniform(1,30), 1)
        medicion = self.ultima_medicion[planta.name()][planta.id] - disminucion
        self.ultima_medicion[planta.name()][planta.id] = round(max(min(medicion, 100), 0),1)
        return self.ultima_medicion[planta.name()][planta.id]

class Tsensor(Sensor):
    def __init__(self):
        super().__init__()

    def medir(self, v):
        v += random.sample([random.uniform(-5,-1),random.uniform(1,5)], 1)[0]
        self.ultima_medicion = round(max(min(v, 50), -30), 1)
        return self.ultima_medicion

class Lsensor(Sensor):
    def __init__(self):
        super().__init__() 

    def medir(self, v):
        v -= random.randint(1,5)
        self.ultima_medicion = round(max(min(v, 100), 0), 1)
        return self.ultima_medicion

In [ ]:
def light(hora: datetime) -> float:
    hour = hora.hour
    minute = hora.minute

    if (6 <= hour <= 11) or (hour == 12 and minute == 0):
        minutes = (hour-6) * 60 + minute
        return round(100*(minutes/((12-6)*60)), 1)
    elif (12 <= hour <= 18) or (hour == 19 and minute == 0):
        minutes = (hour-12) * 60 + minute
        return 100 - round(100*(minutes/((19-12)*60)), 1)
    else:
        return 0

def temp(hora: datetime) -> float:
    return light(hora)*(3/10)+random.uniform(-10,10)

def regar(planta, t: float, l: float, h: float):
    #voy a regar SSI hay luz > 60,
    #la temp > 1 (no puedo regar hielo) y la humedad de la planta 
    #es menor a la mitad del optimo para ella
    if t > 1 and l > 60 and planta.humedad_optima/2 > humedad:
        riego = round(planta.humedad_optima - humedad, 1)
        print(f"\nSe ha regado el/la {planta.name()} {planta.id}, un total de {riego} puntos de humedad")
        print(f"Tenia {planta.humedad_actual} puntos de humedad, las condiciones son {t} grados C y {l} lumens")
        planta.humedad_actual += riego
    
    return planta

In [ ]:
# Defina su invernadero
invernadero = {"Lechuga": list(), "Tomate": list(), "Cilantro": list()}
clock = datetime(year=2020, month=8, day=13, hour=0, minute=0, second=0)
sound = 'tok'

# Defina sus plantas y sensores
print('Materializando Plantas')
for planta in [Lechuga, Tomate, Cilantro]:
    for _ in range(random.randint(3,8)):
        p = planta()
        p.fecha_plantacion = clock
        print(f"Se ha materializado un {p.full_name()} con id {p.id} y tiene {p.puede_cosechar()} {p.fruto()}s")
        invernadero[p.name()].append(p)

print(F"\nEn total se materializaron:\n- {len(invernadero['Lechuga'])} Lechugas\n- {len(invernadero['Tomate'])} Tomates\n- {len(invernadero['Cilantro'])} Cilantros")

s_temp = Tsensor()
s_luz = Lsensor()
s_hum = Hsensor()

print("\nSe han materializado los sensores")

dias_a_simular = 30 # Total de dias que va a simular

print(f"Empezando la simulacion de {dias_a_simular} dias")

for _ in range(dias_a_simular*24):
    # mido los datos
    temperatura = s_temp.medir(temp(clock))
    luz = s_luz.medir(light(clock))
    
    # crecer
    for key in invernadero.keys():
        for planta in invernadero[key]:
            planta.crecer(luz)
            
    # medir humedad y regar
    for k, plantas in invernadero.items():
        for i, planta in enumerate(plantas):
            humedad = s_hum.medir(planta)
            planta.set_humedad_medida(humedad)
            invernadero[k][i] = regar(planta, temperatura, luz, humedad)

    # cosechar
    # no soy vampiro, cosecho solo durante el dia y hora por medio (soy una persona ocupada...)
    if (10 <= clock.hour <= 23) and (clock.hour % 2 == 0): 
        for plantas in invernadero.values():
            for planta in plantas:
                cant = planta.puede_cosechar(clock)
                if cant > 0: 
                    planta.cosechar(cant)
                    print(f"\nSe ha cosechado el/la {planta.name()} {planta.id}:")
                    print(f"Las condiciones son {temperatura} grados C, {luz} lumens y {humedad} puntos de humedad")
                    print(f"Quedo con {planta.puede_cosechar()} {planta.fruto()}s")

    clock += timedelta(hours=1)

    if sound == 'tik':
        print('\ntok', clock)
        sound = 'tok'
    elif sound == 'tok':
        print('\ntik', clock)
        sound = 'tik'

print("\nSimulacion terminada")

for tipo in invernadero:
    for planta in invernadero[tipo]:
        print(f"De {planta.name()} {planta.id} se recolectaron {planta.cant_cosechada} {planta.fruto()}s")

Materializando Plantas
Se ha materializado un Lechuga con id 1 y tiene 0 hojas
Se ha materializado un Lechuga con id 2 y tiene 0 hojas
Se ha materializado un Lechuga con id 3 y tiene 0 hojas
Se ha materializado un Lechuga con id 4 y tiene 0 hojas
Se ha materializado un Lechuga con id 5 y tiene 0 hojas
Se ha materializado un Tomate cherry con id 1 y tiene 0 tomates
Se ha materializado un Tomate normal con id 2 y tiene 0 tomates
Se ha materializado un Tomate cherry con id 3 y tiene 0 tomates
Se ha materializado un Cilantro con id 1 y tiene 0 ramas
Se ha materializado un Cilantro con id 2 y tiene 0 ramas
Se ha materializado un Cilantro con id 3 y tiene 0 ramas
Se ha materializado un Cilantro con id 4 y tiene 0 ramas
Se ha materializado un Cilantro con id 5 y tiene 0 ramas

En total se materializaron:
- 5 Lechugas
- 3 Tomates
- 5 Cilantros

Se han materializado los sensores
Empezando la simulacion de 30 dias

tik 2020-08-13 01:00:00

tok 2020-08-13 02:00:00

tik 2020-08-13 03:00:00

tok 20